# Serial control of AD9910 through Arduino

### Hardware setup

In order to control AD9910 on its official [evaluation board](https://www.analog.com/en/design-center/evaluation-hardware-and-software/evaluation-boards-kits/eval-ad9910.html) through a serial (3-wire SPI) connection, set the board as follows:

1. Disable jumpers `W1`, `W2`, and `W4`. Remove jumpers `W3`, `W5`, `W6`.
2. Connect the following pins to the Arduino's `reset_pin` (see below), and also to ground via a 10k pull-down resistor: in header `U9`, pins `RESET` and `EXT_PWR_DWN`; in `U5`, pin `IO_RESET`.
3. Connect wires for the SPI interface: in header `U5`, pins `SDIO`, `SCLK`, and `CSB` should respectively be connected to the `MISO`, `SCK`, and `CS_pin`. In addition, pin `IO_update` in header `P1` should be connected to the Arduino's `IO_update_pin`.

Note that the above-mentioned pins have the following default values in [`SPI_test.ino`](https://github.com/js216/Arduino-codes/blob/master/SPI_test/SPI_test.ino), as appropriate for a [Teensy 3.6](https://www.pjrc.com/teensy/td_libs_SPI.html) device:

    // SCK = Clock = pin 13
    // MOSI = Data Output = pin 11
    // MISO = Data Input = pin 12
    const int CS_pin = 16;
    const int IO_update_pin = 24;
    const int reset_pin = 2;
    
Since an Arduino is used as a serial-to-SPI link to interface between the computer and the AD9910, it has to be programmed appropriately. See [`SPI_test.ino`](https://github.com/js216/Arduino-codes/blob/master/SPI_test/SPI_test.ino) for example code that is assumed in the following. Other Arduino-like boards might necessitate other pin assignments; see the [official website](https://www.arduino.cc/en/reference/SPI) for details, and adjust the provided [Arduino code](https://github.com/js216/Arduino-codes/blob/master/SPI_test/SPI_test.ino) as appropriate.
    
### Software implementation

Import packages:

In [1]:
import pyvisa

Connect to the Arduino:

In [2]:
rm = pyvisa.ResourceManager()
a = rm.open_resource('ASRL13::INSTR')

Write and test the function that prints the contents of all 23 registers:

In [3]:
def read_registers(arduino):
    print(arduino.query('r'), end='')
    for i in range(23):
        print(arduino.read(), end='')
        
read_registers(a)

Reading register contents:
   0: 00000000,00000000,00000000,00000000,
   1: 00000000,01000000,00001000,00100000,
   2: 00011111,00111111,01000000,00000000,
   3: 00000000,00000000,01111111,01111111,
   4: 11111111,11111111,11111111,11111111,
   5: 11101001,00100000,01100100,10001001,11000101,11100000,
   6: 10000111,00101000,00000101,10101001,10000000,00101000,
   7: 00000000,00000000,00000000,00000000,
   8: 00000000,00000000,
   9: 00000000,00000000,00000000,00000000,
   A: 00000000,00000000,00000000,00000000,
   B: 10100000,10011001,11100100,00011010,00011001,00101010,00000011,11000100,
   C: 11000100,10100000,10110001,10000101,00000011,11000000,10000011,10110000,
   D: 01001110,11010001,00000000,00011100,
   E: 00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,
   F: 00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,
  10: 00000000,00000000,00000000,00000000,00000000,00000000,00000000,00000000,
  11: 00000000,00000000,00000000,00000000,00

Write functions to reset the AD9910, and to write register contents:

In [4]:
def reset(arduino):
    print(arduino.query('0'))
    
def write_register(arduino, reg, data_list, printing=False):
    # send the write command
    arduino.write_raw(
        b"w" \
          + bytes(bytearray([reg])) \
          + bytes(bytearray(data_list))
      )
    
    # print Arduino's return text
    if printing:
        print(arduino.read(), end='')
        print(arduino.read(), end='')

The following writes the register values that cause the AD9910 to output an 85 MHz sine wave for a 1 GHz clock input.

Note in particular the MSB of the second least significant byte in register 0x02, "REFCLK input divider bypass", which, when set to 1, disables the internal clock divider.

The amplitude, phase, and frequency settings of profile 0 are written in the 0x0E register.

In [5]:
write_register(a, 0x00, [0b00000000,0b00000000,0b00000000,0b00000000,])
write_register(a, 0x01, [0b00000000,0b01000000,0b00001000,0b00100000,])
write_register(a, 0x02, [0b00011111,0b00111111,0b11000000,0b00000000,])
write_register(a, 0x03, [0b00000000,0b00000000,0b01111111,0b01111111,])
write_register(a, 0x04, [0b11111111,0b11111111,0b11111111,0b11111111,])
write_register(a, 0x05, [0b11101001,0b00100000,0b01100100,0b10001001,0b11000100,0b11100000,])
write_register(a, 0x06, [0b10000111,0b00101100,0b00000101,0b10101001,0b10000000,0b00101001,])
write_register(a, 0x07, [0b00000000,0b00000000,0b00000000,0b00000000,])
write_register(a, 0x08, [0b00000000,0b00000000,])
write_register(a, 0x09, [0b00000000,0b00000000,0b00000000,0b00000000,])
write_register(a, 0x0A, [0b00000000,0b00000000,0b00000000,0b00000000,])
write_register(a, 0x0B, [0b10100001,0b10011001,0b11000100,0b00011010,0b00011001,0b00101110,0b00000011,0b11000100,])
write_register(a, 0x0C, [0b11000100,0b10101001,0b10110000,0b10000101,0b00000011,0b11000000,0b10000011,0b10110000,])
write_register(a, 0x0D, [0b01001110,0b11010000,0b00000000,0b00011100,])
write_register(a, 0x0E, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x0F, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x10, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x11, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x12, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x13, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x14, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x15, [0b00001000,0b10110101,0b00000000,0b00000000,0b00010101,0b11000010,0b10001111,0b01011100,])
write_register(a, 0x16, [0b00111000,0b11010000,0b01010011,0b11000111,])

Close connection to the Arduino:

In [6]:
a.close()

### Human-friendly wrapper functions

In the following, I provide wrappers for the most commonly used functions of the AD9910.